Each file contains the history of simulation of a Markov Chain.

Assignment: Estimate the context trees through BIC.

# Packages

In [1]:
import itertools # tools for iteration
from collections import Counter # count number of repetitions in list
import copy
import numpy as np
import random
import networkx as nx # graph utilities
import math
from datetime import datetime
from pandas import DataFrame # pandas dataframe
import os # read directory

Clean directory and clone repository with data files

In [2]:
!rm -r sample_data
!rm -r MAE4007tasks
!git clone --branch contextTreeRevision https://github.com/LucasMSpereira/MAE4007tasks

rm: cannot remove 'sample_data': No such file or directory
Cloning into 'MAE4007tasks'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 81 (delta 3), reused 0 (delta 0), pack-reused 72
Unpacking objects: 100% (81/81), 607.23 KiB | 2.17 MiB/s, done.


# Read and organize data

In [3]:
# directory with data
dataDir = './MAE4007tasks/contextTreeData'
# list of dictionaries, each referring to a file
data = []
# read each file and fill 'data'
for fileName in os.listdir(dataDir + "/txtData"):
  # dictionary for current file
  currentData = {}
  data.append(currentData)
  # file name
  data[-1]["fileName"] = fileName
  # open, read and close file
  with open(dataDir + '/txtData/' + fileName, encoding = "utf-8") as f:
    fileData = list(map(int, f.read()))
  # file data as list of integers
  data[-1]["content"] = fileData
  # size of sample
  data[-1]["size"] = len(data[-1]["content"])
  # count occurrences of each value in current file
  data[-1]["valueCount"] = dict(Counter(data[-1]["content"]))
  # unique values in current file
  data[-1]["uniqueValues"] = list(data[-1]["valueCount"].keys())
  data[-1]["uniqueValues"].sort()
  data[-1]["penalty"] = data[-1]["size"] ** (-(len(data[-1]["uniqueValues"]) - 1) / 2)
for d in data:
  for f in d:
    if f != "content":
      print(f"{f}: {d[f]}")
  print()

fileName: OrderEst_ex1.txt
size: 10000
valueCount: {1: 4726, 2: 5274}
uniqueValues: [1, 2]
penalty: 0.01

fileName: OrderEst_ex2.txt
size: 10000
valueCount: {2: 6327, 1: 3673}
uniqueValues: [1, 2]
penalty: 0.01

fileName: OrderEst_ex3.txt
size: 10000
valueCount: {1: 3943, 2: 2989, 3: 3068}
uniqueValues: [1, 2, 3]
penalty: 0.0001

fileName: 52845.txt
size: 30000
valueCount: {1: 13668, 3: 8441, 2: 7891}
uniqueValues: [1, 2, 3]
penalty: 3.3333333333333335e-05



# BIC for context trees

1.   Begin with the full context tree
2.   Starting from leaves:

  2.1.   Define the V value of nodes. In the case of leaves, use the likelihood of its word. In nodes, use the greater value between the
likelihood of its word, and the product of the V values of its immediate successors

  2.2.   Define the binary $\chi$ value of nodes. Assign 1 if: the word's length is less than the maximum (tree depth); and the product of the V values of its immediate successors is greater than the likelihood of the word of the node. Assign 0 otherwise.
3.   Remove nodes (and respective branches) that are dependent on a node whose $\chi = 0$.

\*Assignment limits chain orders to 8

Utilities

In [4]:
# maximum size of tree
treeOrder = 8

def addNodeToTree(nodeID: int, word: tuple, _contextTree: nx.DiGraph, df: DataFrame, fileID: int) -> None:
  """
  Include node in tree. Connect it to successors
  formed by appending each symbol in alphabet, and
  predecessor (word without it's last symbol)
  """
  _contextTree.add_node(nodeID)
  # connect node to neighbors.
  if len(word) > 1: # if word of size at least 2
    _contextTree.add_edge( # connect to predecessor
      IDofWord(word[:-1], df), nodeID
    )
    # if word isn't at lowest level of tree
    if len(word) < treeOrder:
      # connect to successors formed by appending each symbol in alphabet
      for state in data[fileID]["uniqueValues"]:
        _contextTree.add_edge(
          nodeID, IDofWord(word + (state,), df)
        )
  else: # if word of length 1
    # connect to root
    _contextTree.add_edge(-1, nodeID)
    # connect to successors formed by appending each symbol in alphabet
    for state in data[fileID]["uniqueValues"]:
      _contextTree.add_edge(
        nodeID, IDofWord(word + (state,), df)
      )

def calcKhi(word: tuple, fileID: int, _df: DataFrame, _tree: nx.DiGraph):
  """Determine Khi value for current node in context tree"""
  # product of Vs of successors
  Vprod = successorsVprod(_tree, word, _df)
  if len(word) < treeOrder and Vprod > wordLikelihood(word, fileID, _df) * data[fileID]["penalty"]:
    return 1
  else:
    return 0

def calcV(word: tuple, _tree: nx.DiGraph, _df: DataFrame, fileID: int):
  """Determine V value for a node in the context tree"""
  wl = wordLikelihood(word, fileID, _df)
  # if node of word is a leaf
  if isLeaf(_tree, IDofWord(word, _df)):
    return wl * data[fileID]["penalty"]
  else: # if not a leaf
    return max(wl * data[fileID]["penalty"], successorsVprod(_tree, word, _df))

def IDofWord(word: tuple, _occurDataFrame: DataFrame):
  """ID a word's node (same as row in dataframe)"""
  return _occurDataFrame[_occurDataFrame.word == word].index[0]

def isLeaf(tree: nx.DiGraph, nodeID: int):
  """Boolean indicating if node is a leaf"""
  return len(list(tree.successors(nodeID))) == 0

def successorsVprod(tree: nx.DiGraph, _word: tuple, df: DataFrame):
  """Product of V values of a node's immediate successors"""
  successors = tree.successors(IDofWord(_word, df))
  return math.prod([tree.nodes[node]["V"] for node in successors])

def wordCount(occurDataFrame: DataFrame, word: tuple) -> int:
  """Number of occurrences of certain word"""
  return occurDataFrame.at[
    occurDataFrame[occurDataFrame.word == word].index[0],
    "occurrences"
  ]

def wordLikelihood(_word: tuple, fileIndex: int, df: DataFrame):
  """Calculate likelihood of word"""
  likelihood = 1.0
  nCounts = wordCount(df, _word)
  if nCounts == 0:
    return 1
  # iterate in alphabet
  for symbol in data[fileIndex]["uniqueValues"]:
    newWordCount = wordCount(df, _word + (symbol,))
    likelihood *= (newWordCount / nCounts) ** newWordCount
  return likelihood

In [5]:
# loop in files
for f in range(len(data)):
  print(f"""\n******\nFile {f + 1}/{len(data)}   Time: {datetime.now().strftime("%H:%M:%S")}\n******""")
  # directed graph for context tree
  data[f]["contextTree"] = nx.DiGraph()
  contextTree = data[f]["contextTree"]
  # all possible words with size up to treeOrder + 1
  allWords = list(
      itertools.chain.from_iterable(
          list(
              [
                list(
                    itertools.product(
                        data[f]["uniqueValues"],
                        repeat = order
                    )
                ) for order in range(1, treeOrder + 1 + 1)
              ]
          )
      )
  )
  # dictionaries with words, their lengths, and number of occurrences
  occurrenceCount = [{"word": word, "occurrences": 0, "size": len(word)} for word in allWords]
  # organize in pandas dataframe for convenience
  occurrenceCount = DataFrame(occurrenceCount)
  # build basic tree topology
  print("\nBuilding tree topology...")
  for (id, word) in enumerate(allWords):
    if len(word) > 8:
      continue
    # add current word to tree
    addNodeToTree(id, word, contextTree, occurrenceCount, f)
    # set node attributes
    nx.set_node_attributes(contextTree,
        {id:
          {
              "word": ''.join(map(str, word)),
              "occurrences": 0, "khi": -1, "V": -1, "likelihood": -1
          }
        }
    )
  # count occurrences of all words
  print("Counting occurrences of all words...\n")
  for wordSize in range(1, treeOrder + 1): # iterate in possible sizes of words
    print(f"""Word size: {wordSize}    Time: {datetime.now().strftime("%H:%M:%S")}""")
    for position in range(data[f]["size"] - wordSize + 1): # iterate in sequence
      word = data[f]["content"][position : position + wordSize] # current word
      wordID = IDofWord(tuple(word), occurrenceCount) # id of current word
      # increment number of times this word has been seen
      occurrenceCount.at[wordID, "occurrences"] += 1 # update DataFrame
      contextTree.nodes[wordID]["occurrences"] += 1 # update tree
  # determine V and khi values by iterating in possible sizes of
  # words in reversed order, so the tree is updated from bottom to top
  print("\nDetermining V and khi values...\n")
  for wordSize in range(treeOrder, 0, -1):
    print(f"""Tree level: {wordSize}    Time: {datetime.now().strftime("%H:%M:%S")}""")
    # IDs of words of current size
    wordIndices = list(np.where(np.array(list(occurrenceCount['size'] == wordSize)) == True)[0])
    for id in wordIndices: # iterate in words of current size
      currentWord = tuple(map(int, contextTree.nodes[id]["word"]))
      # calculate likelihood
      contextTree.nodes[id]["likelihood"] = wordLikelihood(currentWord, f, occurrenceCount)
      # calculate V
      contextTree.nodes[id]["V"] = calcV(currentWord, contextTree, occurrenceCount, f)
      # calculate khi
      contextTree.nodes[id]["khi"] = calcKhi(currentWord, f, occurrenceCount, contextTree)


******
File 1/4   Time: 19:19:44
******

Building tree topology...
Counting occurrences of all words...

Word size: 1    Time: 19:19:44
Word size: 2    Time: 19:19:56
Word size: 3    Time: 19:20:05
Word size: 4    Time: 19:20:11
Word size: 5    Time: 19:20:18
Word size: 6    Time: 19:20:24
Word size: 7    Time: 19:20:30
Word size: 8    Time: 19:20:37

Determining V and khi values...

Tree level: 8    Time: 19:20:43
Tree level: 7    Time: 19:20:44
Tree level: 6    Time: 19:20:45
Tree level: 5    Time: 19:20:46
Tree level: 4    Time: 19:20:46
Tree level: 3    Time: 19:20:46
Tree level: 2    Time: 19:20:46
Tree level: 1    Time: 19:20:46

******
File 2/4   Time: 19:20:46
******

Building tree topology...
Counting occurrences of all words...

Word size: 1    Time: 19:20:46
Word size: 2    Time: 19:20:53
Word size: 3    Time: 19:20:59
Word size: 4    Time: 19:21:06
Word size: 5    Time: 19:21:12
Word size: 6    Time: 19:21:18
Word size: 7    Time: 19:21:25
Word size: 8    Time: 19:21:32

D

# Prune trees

In [6]:
for f in range(len(data)): # iterate in files
  print(f"File {f + 1}/{len(data)}")
  data[f]["prunedContextTree"] = nx.DiGraph()
  contextTree = data[f]["contextTree"]
  b = True # binary "activating"
  sizeOfLastWord = 0
  for nodeID in nx.dfs_preorder_nodes(contextTree, source = -1):
    if nodeID == -1:
      continue
    if len(contextTree.nodes[nodeID]["word"]) <= sizeOfLastWord:
      b = True
    if not b:
      continue
    # add current word to tree
    # if len()
    addNodeToTree(nodeID, tuple(map(int, contextTree.nodes[nodeID]["word"])), data[f]["prunedContextTree"], occurrenceCount, f)
    # set node attributes
    nx.set_node_attributes(data[f]["prunedContextTree"], {nodeID: contextTree.nodes[nodeID]})
    # data[f]["prunedContextTree"].nodes[nodeID] = contextTree.nodes[nodeID]
    if contextTree.nodes[nodeID]["khi"] == 0 and len(contextTree.nodes[nodeID]["word"]) < treeOrder:
      b = False
      sizeOfLastWord = len(contextTree.nodes[nodeID]["word"])

File 1/4
File 2/4
File 3/4
File 4/4


#Process order

Estimate process order by size of longest word in pruned trees.

In [7]:
for f in range(len(data)):
  prunedTree = data[f]["prunedContextTree"]
  wordSizes = [] # sizes of words with khi = 0
  # iterate in depth-first seach nodes
  for node in nx.dfs_preorder_nodes(prunedTree, source = -1):
    if node == -1:
      continue
    try:
      if prunedTree.nodes[node]["khi"] == 0:
        wordSizes.append(len(prunedTree.nodes[node]["word"]))
    except:
      continue
  print("Estimated order of markov process in file {:} is {:n}".format(data[f]["fileName"], max(wordSizes)))

Estimated order of markov process in file OrderEst_ex1.txt is 8
Estimated order of markov process in file OrderEst_ex2.txt is 8
Estimated order of markov process in file OrderEst_ex3.txt is 1
Estimated order of markov process in file 52845.txt is 1
